In [1]:
pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import re
import os
import numpy as np
import pandas as pd
import faiss
import openpyxl

In [4]:
content = pd.read_excel("faiss_terms/content.xlsx")

In [5]:
content.tail()

,item,embedding
3017,gutenberg bible,[ 2.33509322e-03 4.54344414e-02 1.25965197e-...
3018,shrunken head,[ 1.99726652e-02 -6.52201176e-02 -2.89458558e-...
3019,canopic jar,[ 5.88798299e-02 -8.60870350e-03 -1.21568469e-...
3020,ushabti figurin,[ 7.17326952e-03 -1.16594331e-02 -2.85282396e-...
3021,Struggle,[ 2.41651442e-02 1.04914837e-01 -1.30357482e-...


In [6]:
def string_parser(s: str):

    cleaned_s = s.strip().replace("]",'').replace("[",'')

    return [float(item) for item in re.split(r'\s+', cleaned_s) if item]

In [7]:
content_embeddings = content["embedding"].apply(string_parser).to_list()

In [8]:
type(content_embeddings)

list

In [9]:
print(type(content_embeddings[0][0]))

<class 'float'>


In [10]:
arr2d = np.array(content_embeddings, dtype=np.float32)

In [11]:
num_embeddings, embedding_dim = arr2d.shape

In [12]:
arr2d.shape

(3022, 768)

In [13]:
num_embeddings

3022

In [14]:
content_k = int(4 * np.sqrt(num_embeddings))

In [15]:
content_k

219

In [16]:
quantizer = faiss.IndexFlatL2(embedding_dim)
index = faiss.IndexIVFFlat(quantizer, embedding_dim, content_k)

In [17]:
index.train(arr2d)

In [18]:
index.make_direct_map()

In [19]:
index.add(arr2d)

In [20]:
faiss.write_index(index, "content.index")

In [24]:
## Style ##

In [25]:
style = pd.read_excel("faiss_terms/stylistic.xlsx")

In [26]:
style.head()

,item,embedding
0,Abstract Expressionism,[ 3.58736180e-02 7.82950595e-02 -4.45399210e-...
1,Academic Art,[ 4.31802236e-02 6.17554598e-02 -2.78073996e-...
2,Action Painting,[-2.92749703e-02 -3.50885317e-02 -4.37009288e-...
3,Aestheticism,[ 1.37087433e-02 8.78865868e-02 -2.62958333e-...
4,Afrofuturism,[ 2.83689480e-02 4.59528118e-02 5.46957506e-...


In [27]:
style_embeddings = style["embedding"].apply(string_parser).to_list()

In [28]:
style_np = np.array(style_embeddings, dtype=np.float32)

In [29]:
num_style_embeddings, style_embeddings_ndim = style_np.shape

In [30]:
style_embeddings_ndim

768

In [31]:
style_k = int(4 * np.sqrt(num_style_embeddings))

In [32]:
style_k

137

In [33]:
quantizer = faiss.IndexFlatL2(style_embeddings_ndim)
style_index = faiss.IndexIVFFlat(quantizer, style_embeddings_ndim, style_k)

In [34]:
style_index.train(style_np)

In [35]:
style_index.make_direct_map()

In [36]:
style_index.add(style_np)

In [37]:
faiss.write_index(style_index, "style.index")

In [38]:
execs = pd.read_excel("faiss_terms/execution.xlsx")

In [39]:
execs.head()

,item,embedding
0,golden hour,[-2.09865365e-02 4.55007004e-03 1.06995488e-...
1,blue hour,[-2.56597456e-02 -4.76618074e-02 8.15100037e-...
2,magic hour,[-1.46740312e-02 -2.49671601e-02 -5.43604186e-...
3,sunrise,[-2.81988289e-02 9.31332782e-02 -1.80892125e-...
4,sunset,[-2.87785847e-02 4.08789963e-02 -1.06120417e-...


In [40]:
execs_embeddings = execs["embedding"].apply(string_parser).to_list()

In [41]:
execs_np = np.array(execs_embeddings, dtype=np.float32)

In [42]:
num_execs, execs_dim = execs_np.shape

In [43]:
exec_k = int(4 * np.sqrt(num_execs))

In [44]:
quantizer = faiss.IndexFlatL2(execs_dim)
exec_index = faiss.IndexIVFFlat(quantizer, execs_dim, exec_k)

In [45]:
exec_index.train(execs_np)

In [46]:
exec_index.make_direct_map()

In [47]:
exec_index.add(execs_np)

In [48]:
faiss.write_index(exec_index, "execution.index")